In [1]:
### Train and evaluate models using only the color features ###

In [2]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from lazypredict.Supervised import LazyClassifier

from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

In [3]:
# load data and drop unwanted columns
data = pd.read_csv('../dataset_versions/50neither_vs_mel_keratosis.csv')
features_target = data.drop(data.iloc[:,15:], axis=1)
features_target.drop(features_target.columns[[1,2]], axis=1)
features_target.columns

Index(['image_id', 'age_approximate', 'sex', 'melanoma',
       'seborrheic_keratosis', 'neither', 'red_mode', 'green_mode',
       'blue_mode', 'red_median', 'green_median', 'blue_median', 'red_iqr',
       'green_iqr', 'blue_iqr'],
      dtype='object')

In [4]:
for i,u in enumerate(features_target['image_id']):
    if features_target.loc[i,'melanoma'] == 0:
        if features_target.loc[i,'seborrheic_keratosis'] == 1:
            features_target.loc[i,'melanoma'] = 2
    else:
        continue
        
features_target.drop(columns=['seborrheic_keratosis', 'neither'], inplace=True)
features_target

,image_id,age_approximate,sex,melanoma,red_mode,green_mode,blue_mode,red_median,green_median,blue_median,red_iqr,green_iqr,blue_iqr
0,ISIC_0000001,30,female,0,87,53,46,116,72,55,86,58,40
1,ISIC_0000002,60,female,1,162,120,114,161,131,131,26,52,67
2,ISIC_0000004,80,male,1,192,132,171,192,128,157,36,44,55
3,ISIC_0000006,25,female,0,213,123,100,187,152,130,38,76,101
4,ISIC_0000009,30,female,0,187,122,90,194,138,100,35,60,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,ISIC_0015220,75,male,2,120,36,20,136,57,46,46,47,51
1310,ISIC_0015233,60,male,2,71,124,101,115,80,63,97,73,59
1311,ISIC_0015260,55,female,2,76,21,13,77,40,24,65,51,34
1312,ISIC_0015284,75,male,1,136,19,6,143,80,58,52,76,85


In [5]:
# train and test split
X = features_target.iloc[:,6:]
y = features_target[['melanoma']]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
X

,blue_mode,red_median,green_median,blue_median,red_iqr,green_iqr,blue_iqr
0,46,116,72,55,86,58,40
1,114,161,131,131,26,52,67
2,171,192,128,157,36,44,55
3,100,187,152,130,38,76,101
4,90,194,138,100,35,60,58
...,...,...,...,...,...,...,...
1309,20,136,57,46,46,47,51
1310,101,115,80,63,97,73,59
1311,13,77,40,24,65,51,34
1312,6,143,80,58,52,76,85


In [7]:
# Preliminary model training and evaluation with LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)

100%|███████████████████████████████████████████| 29/29 [00:03<00:00,  8.75it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
QuadraticDiscriminantAnalysis      0.58               0.58    None      0.54   
NuSVC                              0.59               0.51    None      0.57   
ExtraTreesClassifier               0.55               0.49    None      0.52   
GaussianNB                         0.51               0.48    None      0.42   
LabelSpreading                     0.52               0.48    None      0.52   
NearestCentroid                    0.47               0.48    None      0.45   
AdaBoostClassifier                 0.54               0.48    None      0.48   
LabelPropagation                   0.52               0.47    None      0.51   
XGBClassifier                      0.53               0.47    None      0.51   
LogisticRegression                 0.56               0.47    None      0.48   
RandomForestClassifier             0.53 

In [8]:
# Build the MPL
mlp_mk = MLPClassifier()


# Hyperparameters to test
params = {
    'activation': ['tanh', 'relu','identity','logistic'],
    'solver': ['sgd', 'adam','sgd'],
    'alpha': [0.0001, 0.001, 0.05],
    'learning_rate': ['constant','adaptive', 'invscaling'],
}

# scale data
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# train and evaluate the models
neural_net = GridSearchCV(mlp_mk, params, n_jobs=-1, cv=5)
neural_net.fit(X_train, y_train) 
print("Best parameter (CV score=%0.3f):" % neural_net.best_score_)
print(neural_net.best_params_)
# test the model on new data                
y_true, y_pred = y_test, neural_net.predict(X_test)
print('Results on the test set: ')
print(classification_report(y_true, y_pred))

/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: Convergen

/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConv

/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/

/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d a

/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d a

...

/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConv

/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d a

/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConv

/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ricardo/Documents/MEB/AA/aaenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConv

Best parameter (CV score=0.590):
{'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'solver': 'sgd'}
Results on the test set: 
              precision    recall  f1-score   support

           0       0.54      0.85      0.66       164
           1       0.30      0.03      0.05       102
           2       0.51      0.48      0.49        63

    accuracy                           0.53       329
   macro avg       0.45      0.45      0.40       329
weighted avg       0.46      0.53      0.44       329

